# small slim tutorial

In [3]:
import os

## function to run slim simulations

In [6]:
def slim_script(script, name, quiet=False):
    logfile = name + ".log"
    out = os.system("echo '" + script + "' | slim > " + logfile + " 2>&1")
    if not quiet:
        with open(logfile, "r") as log:
            print(log.read())
    if out != 0:
        print("An error occurred.")
    return out, logfile

## basic WF model 

In [7]:
basic_WF = """
// set up a simple neutral simulation
initialize()
{
    // set the overall mutation rate
    initializeMutationRate(1e-7);

    // m1 mutation type: neutral
    initializeMutationType("m1", 0.5, "f", 0.0);
    // m2 mutation type: beneficial
    initializeMutationType("m2", 0.5, "f", 0.01);

    // g1 genomic element type: uses m1 or m2 with equal prob for mutations
    initializeGenomicElementType("g1", c(m1,m2), c(1.0,1.0));

    // uniform chromosome of length 100 kb
    initializeGenomicElement(g1, 0, 99999);
   
    // uniform recombination along the chromosome
    initializeRecombinationRate(1e-8);
}

// create a population of 500 individuals
1 {
    sim.addSubpop("p1", 500);
}

// run to generation 10000
10000 {
    sim.simulationFinished();
}
"""

out, logfile = slim_script(basic_WF, "basic_WF")

SLiM version 3.2, built Nov  6 2018 09:41:03.

SLiM is a product of the Messer Lab, http://messerlab.org/
Copyright 2013-2018 Philipp Messer.  All rights reserved.

By Benjamin C. Haller, http://benhaller.com/, and Philipp Messer.

---------------------------------------------------------------------------------

To cite SLiM in publications please use:

Haller, B.C., and Messer, P.W. (2017). SLiM 2: Flexible, Interactive Forward
Genetic Simulations. Molecular Biology and Evolution 34(1), 230-240.
DOI: 10.1093/molbev/msw211

---------------------------------------------------------------------------------

SLiM home page: http://messerlab.org/slim/
slim-announce mailing list: https://groups.google.com/d/forum/slim-announce
slim-discuss mailing list: https://groups.google.com/d/forum/slim-discuss

---------------------------------------------------------------------------------

SLiM is free software: you can redistribute it and/or modify it under the terms
of the GNU General Public Lic

In [10]:
out

0

## basic non-WF

In [11]:
basic_nonWF = """
initialize()
{
    // since the model will be non-Wright-Fisher, we need to say so
    initializeSLiMModelType("nonWF");
    // genome is the same as above
    initializeMutationRate(1e-7);
    initializeMutationType("m1", 0.5, "f", 0.0);
    initializeGenomicElementType("g1", m1, 1.0);
    initializeGenomicElement(g1, 0, 99999);
    initializeRecombinationRate(1e-8);    
}

reproduction() {
    subpop.addCrossed(individual, subpop.sampleIndividuals(1));
}

1 early() {
    sim.addSubpop("p1", 10);
}

1: {
    catn(sim.generation + " : population size : " + p1.individualCount);
}

10 {
    sim.simulationFinished();
}
"""
out, logfile = slim_script(basic_nonWF, "basic_nonWF", quiet=False)

SLiM version 3.2, built Nov  6 2018 09:41:03.

SLiM is a product of the Messer Lab, http://messerlab.org/
Copyright 2013-2018 Philipp Messer.  All rights reserved.

By Benjamin C. Haller, http://benhaller.com/, and Philipp Messer.

---------------------------------------------------------------------------------

To cite SLiM in publications please use:

Haller, B.C., and Messer, P.W. (2017). SLiM 2: Flexible, Interactive Forward
Genetic Simulations. Molecular Biology and Evolution 34(1), 230-240.
DOI: 10.1093/molbev/msw211

---------------------------------------------------------------------------------

SLiM home page: http://messerlab.org/slim/
slim-announce mailing list: https://groups.google.com/d/forum/slim-announce
slim-discuss mailing list: https://groups.google.com/d/forum/slim-discuss

---------------------------------------------------------------------------------

SLiM is free software: you can redistribute it and/or modify it under the terms
of the GNU General Public Lic